In [1]:
%reload_ext autoreload
%autoreload 2

In [40]:
import pandas as pd
from dotenv import load_dotenv
import datasets

load_dotenv(override=True)
pd.set_option("max_colwidth", None)

OUTPUT_DIR = "output"

In [41]:
from huggingface_hub import login
import os

login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful


In [42]:
eval_ds = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all")["validation"]
eval_ds = eval_ds.rename_columns(
    {"Question": "question", "Final answer": "true_answer", "Level": "task"}
)
eval_df = pd.DataFrame(eval_ds)

Using the latest cached version of the module from /Users/aymeric/.cache/huggingface/modules/datasets_modules/datasets/gaia-benchmark--GAIA/ec492fe4320ee795b1aed6bb46229c5f693226b0f1316347501c24b4baeee005 (last modified on Tue May 28 10:04:32 2024) since it couldn't be found locally at gaia-benchmark/GAIA, or remotely on the Hugging Face Hub.


In [43]:
pd.Series(eval_ds["task"]).value_counts()

2    86
1    53
3    26
Name: count, dtype: int64

# 1. Load all results

In [65]:
import glob

answer_file_path = f"{OUTPUT_DIR}/validation/answers.jsonl"

result_df = pd.concat(
    [
        pd.read_json(f, lines=True)
        for f in glob.glob(f"{OUTPUT_DIR}/validation/*.jsonl")
        if "answers.jsonl" not in f
    ]
)
result_df = result_df.drop(columns=["start_time", "end_time"])
result_df.to_json(answer_file_path, lines=True, orient="records")

In [66]:
from scripts.evaluation.gaia_scorer import question_scorer, check_close_call
import re
from collections import Counter

result_df["is_correct"] = result_df.apply(
    lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1
)
result_df["is_near_correct"] = result_df.apply(
    lambda x: check_close_call(x["prediction"], x["true_answer"], x["is_correct"]),
    axis=1,
)

result_df["count_steps"] = result_df["intermediate_steps"].apply(len)


def find_attachment(question):
    matches = eval_df.loc[
        eval_df["question"].apply(lambda x: x in question), "file_name"
    ]

    if len(matches) == 0:
        return "Not found"
    file_path = matches.values[0]

    if isinstance(file_path, str) and len(file_path) > 0:
        return file_path.split(".")[-1]
    else:
        return "None"


result_df["attachment_type"] = result_df["question"].apply(find_attachment)


def extract_tool_calls(code):
    regex = r"\b(\w+)\("
    function_calls = [el for el in re.findall(regex, code) if el.islower()]

    function_call_counter = Counter(function_calls)
    return function_call_counter


def sum_tool_calls(steps):
    total_count = Counter()
    for step in steps:
        if "llm_output" in step:
            total_count += extract_tool_calls(step["llm_output"])

    return total_count


result_df["tool_calls"] = result_df["intermediate_steps"].apply(sum_tool_calls)

/Users/aymeric/Documents/Code/GAIA/scripts/evaluation/gaia_scorer.py:54: UserWarning:

Answer lists have different lengths, returning False.



In [67]:
def get_thoughts(x):
    try:
        output = x[0]["task"]
        for y in x[1:]:
            try:
                if "observation" in y:
                    output += y["llm_output"] + "\nObservation:" + y["observation"]
                else:
                    output += y["llm_output"] + "\Error:" + str(y["error"])
            except:
                pass
        return output
    except:
        return None


result_df["thoughts"] = result_df["intermediate_steps"].apply(lambda x: get_thoughts(x))

# 2. Inspect specific runs

In [68]:
gpt4o = "react_code_gpt4o_23-june_planning2_newprompt5"
claude = "react_code_claude_28-june_planning2_newprompt5"
list_versions = [gpt4o, claude]

# submission_selection_name = "react_code_llama3-70b_02-05_full-gaia-validation-code"
sel_df = result_df.loc[
    (result_df["agent_name"].isin(list_versions))
    # & (~result_df["question"].isin(UNSOLVED_QUESTIONS))
].reset_index(drop=True)
display(sel_df["agent_name"].value_counts())
sel_df = sel_df.drop_duplicates(subset=["agent_name", "question"])
display(sel_df.groupby("agent_name")[["task"]].value_counts())
print("Total length:", len(sel_df), "- is complete:", len(sel_df) == 165)
# assert sel_df["question"].value_counts().max() == len(list_versions), "Some questions are duplicate!"

agent_name
react_code_gpt4o_23-june_planning2_newprompt5     165
react_code_claude_28-june_planning2_newprompt5     16
Name: count, dtype: int64

agent_name                                      task
react_code_claude_28-june_planning2_newprompt5  2        9
                                                1        7
react_code_gpt4o_23-june_planning2_newprompt5   2       86
                                                1       53
                                                3       26
Name: count, dtype: int64

Total length: 181 - is complete: False


In [69]:
from scripts.evaluation.unsolved_questions import UNSOLVED_QUESTIONS

sel_df.loc[
    (sel_df["is_correct"] == False) & (sel_df["is_near_correct"] == True),
    ["question", "prediction", "true_answer"],
]

,question,prediction,true_answer


In [70]:
display("Average score:", sel_df.groupby("agent_name")[["is_correct"]].mean().round(3))
display(
    sel_df.groupby(["agent_name", "task"])[
        ["is_correct", "is_near_correct", "count_steps", "question"]
    ]
    .agg(
        {
            "is_correct": "mean",
            "is_near_correct": "mean",
            "count_steps": "mean",
            "question": "count",
        }
    )
    .rename(columns={"question": "count"})
)

'Average score:'

,is_correct
agent_name,
react_code_claude_28-june_planning2_newprompt5,0.312
react_code_gpt4o_23-june_planning2_newprompt5,0.442


is_correct  \
agent_name                                     task               
react_code_claude_28-june_planning2_newprompt5 1       0.142857   
                                               2       0.444444   
react_code_gpt4o_23-june_planning2_newprompt5  1       0.584906   
                                               2       0.430233   
                                               3       0.192308   

                                                     is_near_correct  \
agent_name                                     task                    
react_code_claude_28-june_planning2_newprompt5 1            0.142857   
                                               2            0.444444   
react_code_gpt4o_23-june_planning2_newprompt5  1            0.584906   
                                               2            0.430233   
                                               3            0.192308   

                                                     count_steps  count  
agent_name                                     task                      
react_code_claude_28-june_planning2_newprompt5 1       10.714286      7  
                                               2       12.111111      9  
react_code_gpt4o_23-june_planning2_newprompt5  1        6.566038     53  
                                               2        9.360465     86  
                                               3       11.576923     26

In [71]:
import plotly.express as px

cumulative_df = (
    (
        sel_df.groupby("agent_name")[["is_correct", "is_near_correct"]]
        .expanding(min_periods=1, axis=0, method="single")
        .agg({"is_correct": "mean", "is_near_correct": "count"})
        .reset_index()
    )
    .copy()
    .rename(columns={"is_near_correct": "index"})
)
cumulative_df["index"] = cumulative_df["index"].astype(int) - 1


def find_question(row):
    try:
        res = sel_df.loc[sel_df["agent_name"] == row["agent_name"], "question"].iloc[
            row["index"]
        ][:50]
        return res
    except Exception as e:
        return ""


cumulative_df["question"] = cumulative_df.apply(find_question, axis=1)
# cumulative_df["question"] = [el[:50] for el in sel_df["question"].values]


# cumulative_df["is_correct"] = cumulative_df["is_correct"] * (165 - 68) / 165


px.line(
    cumulative_df,
    color="agent_name",
    x="index",
    y="is_correct",
    hover_data="question",
)

In [14]:
sel_df.loc[
    (sel_df["agent_name"] == newprompt5) & (sel_df["question"].str.contains("Eva")),
    "question",
]

763    Eva Draconis has a personal website which can be accessed on her YouTube page. What is the meaning of the only symbol seen in the top banner that has a curved line that isn't a circle or a portion of a circle? Answer without punctuation.
Name: question, dtype: object

In [15]:
unsolved_questions = []


def was_question_solved(question):
    for agent in sel_df["agent_name"].unique():
        subselection = sel_df.loc[
            (sel_df["agent_name"] == agent) & (sel_df["question"] == question)
        ]
        if len(subselection) > 0 and subselection["is_correct"].iloc[0] == 1:
            return True
    return False


for question in sel_df.loc[sel_df["agent_name"] == replan, "question"].unique():
    if not was_question_solved(question):
        unsolved_questions.append(question)

len(unsolved_questions)

0

# 3. Dive deeper into one run

In [16]:
sel_df = result_df.loc[result_df["agent_name"] == newprompt5]
print(len(sel_df))

165


### Count errors

In [17]:
import numpy as np

error_types = [
    "AgentParsingError",
    "AgentExecutionError",
    "AgentMaxIterationsError",
    "AgentGenerationError",
]
sel_df[error_types] = 0
sel_df["Count steps"] = np.nan


def count_errors(row):
    if isinstance(row["intermediate_steps"], list):
        row["Count steps"] = len(row["intermediate_steps"])
        for step in row["intermediate_steps"]:
            if isinstance(step, dict) and "error" in step:
                try:
                    row[str(step["error"]["error_type"])] += 1
                except:
                    pass
    return row


sel_df = sel_df.apply(count_errors, axis=1)

/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/ipykernel_69566/2783934181.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/ipykernel_69566/2783934181.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/ipykernel_69566/2783934181.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [18]:
import plotly.express as px


aggregate_errors = (
    sel_df.groupby(["is_correct"])[error_types + ["Count steps"]]
    .mean()
    .reset_index()
    .melt(id_vars=["is_correct"])
)

fig = px.bar(
    aggregate_errors,
    y="value",
    x="variable",
    color="is_correct",
    labels={
        "agent_name": "<b>LLM Engine</b>",
        "task": "<b>Level</b>",
        "aggregate_score": "<b>Performance</b>",
        "value": "<b>Average count</b>",
        "eval_score_GPT4": "<b>Score</b>",
    },
)
fig.update_layout(
    height=500,
    width=800,
    barmode="group",
    bargroupgap=0.0,
)
fig.update_traces(textposition="outside")
fig.write_image("figures/aggregate_errors.png", scale=3)
fig.show()

### Count tool calls

In [19]:
tools_calls = pd.DataFrame.from_records(sel_df["tool_calls"].values).fillna(0)

# Exclude the tools that were not used enough
tools_calls = tools_calls.loc[:, tools_calls.sum() > 10]

# Sort the columns by the sum of the values
tools_calls = tools_calls[tools_calls.sum().sort_values(ascending=False).index]
display(tools_calls)
sel_with_calls = pd.concat(
    [sel_df[["question", "is_correct", "task"]], tools_calls], axis=1
)
sel_with_calls = (
    sel_with_calls.drop("question", axis=1).groupby(["is_correct", "task"]).mean()
)
# sel_with_calls = sel_with_calls.melt(id_vars=['question', 'is_correct', 'task'], var_name="tool", value_name='count')

,print,ask_search_agent,final_answer,len,range,inspect_file_as_text,set,append,visualizer,sum,...,generate_prefixes,check_qualification,search_birthdate,isin,set_piece_at,f,lower,get,sort,abs
0,8.0,3.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,6.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,3.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
163,5.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
sel_with_calls = sel_with_calls.reset_index().melt(
    id_vars=["is_correct", "task"], var_name="tool", value_name="average_count"
)

In [21]:
import plotly.express as px

fig = px.bar(
    sel_with_calls,
    x="tool",
    y="average_count",
    color="is_correct",
    facet_row="task",
    labels={
        "agent_name": "<b>Agent variant</b>",
        "task": "<b>Level</b>",
        "aggregate_score": "<b>Performance</b>",
        "eval_score_GPT4": "<b>Score</b>",
        "agent_type": "<b>Agent type</b>",
        "average_count": "<b>Average #calls per run</b>",
    },
)
fig.update_layout(
    barmode="group",
    height=800,
    width=1000,
    title="<b>" + "</b>",
)

### Inspect result by file extension type

In [43]:
display(
    sel_df.groupby(["agent_name", "attachment_type"])[
        ["is_correct", "count_steps", "question"]
    ].agg({"is_correct": "mean", "count_steps": "mean", "question": "count"})
)

is_correct  \
agent_name                                    attachment_type               
react_code_gpt4o_23-june_planning2_newprompt5 None               0.436508   
                                              csv                0.000000   
                                              docx               0.000000   
                                              jpg                0.000000   
                                              jsonld             0.000000   
                                              mp3                1.000000   
                                              pdb                0.000000   
                                              pdf                0.333333   
                                              png                0.125000   
                                              pptx               1.000000   
                                              py                 1.000000   
                                              txt                0.000000   
                                              xlsx               0.615385   
                                              zip                0.500000   

                                                               count_steps  \
agent_name                                    attachment_type                
react_code_gpt4o_23-june_planning2_newprompt5 None                9.166667   
                                              csv                17.000000   
                                              docx                9.000000   
                                              jpg                 9.500000   
                                              jsonld             16.000000   
                                              mp3                 8.333333   
                                              pdb                 7.000000   
                                              pdf                 5.666667   
                                              png                 6.750000   
                                              pptx                4.000000   
                                              py                  4.000000   
                                              txt                 6.000000   
                                              xlsx                7.538462   
                                              zip                15.000000   

                                                               question  
agent_name                                    attachment_type            
react_code_gpt4o_23-june_planning2_newprompt5 None                  126  
                                              csv                     1  
                                              docx                    1  
                                              jpg                     2  
                                              jsonld                  1  
                                              mp3                     3  
                                              pdb                     1  
                                              pdf                     3  
                                              png                     8  
                                              pptx                    1  
                                              py                      1  
                                              txt                     1  
                                              xlsx                   13  
                                              zip                     2

In [44]:
# Inspect specific file types
# sel_df.loc[
#     sel_df["attachment_type"].isin(["pdb", "docx", "csv"]),
#     [
#         "attachment_type",
#         "question",
#         "prediction",
#         "true_answer",
#         "is_correct",
#         "thoughts",
#     ],
# ]

# 4. Ensembling methods

### 4.1 Simple retry mechanism

In [54]:
first_run_gpt4 = result_df.loc[result_df["agent_name"] == newprompt5].copy()
second_run_gpt4 = result_df.loc[result_df["agent_name"] == noanchorplan].copy()


def replace_answer_if_incomplete(row, result_df_replacement):
    try:
        if (
            "Unable to determine" in row["intermediate_steps"]
            or "AgentMaxIterationsError" in str(row["intermediate_steps"])
            # or "AgentExecutionError" in str(row["intermediate_steps"])
            # or "AgentGenerationError" in str(row["intermediate_steps"])
            or "Error in generating final llm output" in str(row["intermediate_steps"])
        ):
            matching_answer = result_df_replacement.loc[
                (result_df_replacement["question"] == row["question"]), "prediction"
            ].values[0]
            print("replaced")
            gold_answer = matching_answer
        else:
            gold_answer = row["prediction"]
    except:
        gold_answer = row["prediction"]
    return gold_answer


combined_gpt4 = first_run_gpt4.copy()
combined_gpt4["prediction"] = combined_gpt4.apply(
    lambda x: replace_answer_if_incomplete(x, second_run_gpt4), axis=1
)

combined_gpt4["is_correct"] = combined_gpt4.apply(
    lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1
)

replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced
replaced


In [55]:
print("First run:")
print(first_run_gpt4.groupby(["task"])["is_correct"].mean())
print(first_run_gpt4["is_correct"].mean())

print("Second run:")
print(second_run_gpt4.groupby(["task"])["is_correct"].mean())
print(second_run_gpt4["is_correct"].mean())

print("Combined run:")
print(combined_gpt4.groupby(["task"])["is_correct"].mean())
print(combined_gpt4["is_correct"].mean())

First run:
task
1    0.566038
2    0.418605
3    0.200000
Name: is_correct, dtype: float64
0.4329268292682927
Second run:
task
1    0.528302
2    0.372093
3    0.200000
Name: is_correct, dtype: float64
0.39634146341463417
Combined run:
task
1    0.566038
2    0.395349
3    0.160000
Name: is_correct, dtype: float64
0.4146341463414634


### 4.2 Ideal ensembling

In [87]:
third_run = result_df.loc[result_df["agent_name"] == noanchorplan].copy()
INCLUDE_THIRD_RUN = False


# test ideal ensembling
def score_best_both(row, result_df_replacement):
    try:
        if row["is_correct"]:
            return True

        else:
            matching_answer = result_df_replacement.loc[
                (result_df_replacement["question"] == row["question"])
            ].iloc[0]
            if matching_answer["is_correct"]:
                return True
            else:
                return False
    except:
        return row["is_correct"]


combined_gpt4 = first_run_gpt4.copy()
combined_gpt4["is_correct"] = combined_gpt4.apply(
    lambda x: score_best_both(x, second_run_gpt4), axis=1
)
if INCLUDE_THIRD_RUN:
    combined_gpt4["is_correct"] = combined_gpt4.apply(
        lambda x: score_best_both(x, third_run), axis=1
    )
print("Ideal combined run:")
print(combined_gpt4.groupby(["task"])["is_correct"].mean())
print(combined_gpt4["is_correct"].mean())

Ideal combined run:
task
1    0.641509
2    0.465116
3    0.240000
Name: is_correct, dtype: float64
0.4878048780487805


In [1]:
function = """
import numpy as np
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w
"""
task = f"""Here is a basic function that I've devised.

Function:
{function}

Please do the following points in order:
1. Understand what this function does and how to use it
2. Generate a few test functions as python files
3. Run the tests using your Python interpreter tool and make sure the tests pass
4. Return to me the complete code for the test functions.

MAKE SURE YOU HAVE RUN THE TESTS WITH SOME EXAMPLE VARIABLES! ELSE IT WILL BE COUNTED AS A FAILURE

Example:

Function:
def get_even_numbers(numbers):
    even_numbers = []
    for number in numbers:
        if number % 2 == 0:
            even_numbers.append(number)
    return even_numbers

Test functions generated:
def test_get_even_numbers():
    assert get_even_numbers([1, 2, 3, 4, 5]) == [2, 4]

Running the test functions:
running_tests_code_snippet = \"\"\"
def test_get_even_numbers():
    assert get_even_numbers([1, 2, 3, 4, 5]) == [2, 4]

test_get_even_numbers()
\"\"\"
python_interpreter(running_tests_code_snippet)

=> Output: no error.

So we can return the function:
return_snippet = \"\"\"
def test_get_even_numbers():
    assert get_even_numbers([1, 2, 3, 4, 5]) == [2, 4]
\"\"\"

final_answer(return_snippet)

Now proceed!"""

from transformers import ReactCodeAgent, HfEngine, Tool
from transformers.agents import PythonInterpreterTool
from huggingface_hub import login

llm_engine = HfEngine(model="meta-llama/Meta-Llama-3-70B-Instruct")


agent = ReactCodeAgent(
    llm_engine=llm_engine,
    tools=[PythonInterpreterTool(authorized_imports=["numpy"])],
    additional_authorized_imports=["numpy"],
    planning_interval=10,
)

agent.run(task)

/Users/aymeric/venvs/disposable/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
======== New task ========
Here is a basic function that I've devised.

Function:

import numpy as np
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w


Please do the following points in order:
1. Understand what this function does and how to use it
2. Generate a few test functions as python files
3. Run the tests using your Python interpreter tool and make sure the tests pass
4. Return to me the complete code for the test functions.

MAKE SURE YOU HAVE RUN THE TESTS WITH SOME EXAMPLE VARIABLES! ELSE IT WILL BE COUNTED AS A FAILURE

Example:

Function:
def get_even_numbers(numbers):
    even_numbers = []
    for number in numbers:
        if number % 2 == 0:
            even_numbers.append(number)

KeyboardInterrupt: 

In [3]:
from transformers.agents import ReactCodeAgent, Tool

agent = ReactCodeAgent(tools=[])

final_answer_tool = agent._toolbox.tools["final_answer"]


class FinalAnswerToolWithOutputParser(Tool):
    name = "final_answer"
    description = "Provides a final answer to the given problem"
    inputs = {
        "answer": {"type": "text", "description": "The final answer to the problem"}
    }
    output_type = "any"

    def forward(self, answer):
        assert isinstance(answer, list)
        return answer


final_answer_tool = FinalAnswerToolWithOutputParser()

agent._toolbox.update_tool(final_answer_tool)

res = agent.run("Return me a list")

======== New task ========
Return me a list
==== Agent is executing the code below:
import random
my_list = list_generator(length=10)
print(my_list)
====
Failed while trying to execute the code above, due to the following error:
EXECUTION FAILED:
Evaluation stopped at line 'my_list = list_generator(length=10)' because of the following error:
It is not permitted to evaluate other functions than the provided tools or imported functions (tried to execute list_generator).
Traceback (most recent call last):
  File "/Users/aymeric/Documents/Code/original_transformers/transformers/src/transformers/agents/python_interpreter.py", line 834, in evaluate_python_code
    result = evaluate_ast(node, state, tools, authorized_imports)
  File "/Users/aymeric/Documents/Code/original_transformers/transformers/src/transformers/agents/python_interpreter.py", line 680, in evaluate_ast
    return evaluate_assign(expression, state, tools)
  File "/Users/aymeric/Documents/Code/original_transformers/transformer